### Add individual entries to the DB

First, add the entries from another DB

In [1]:
from ase.db import connect
from pathlib import Path

# Read and generate an atoms entry from the NEB run. Write/Update it to the DB.

new_db_path = 'structures/hexag_perovs_strained.db'
old_db_path = 'structures/hexag_perovs_wdiscards.db'

#db_path = shutil.copy(new_db_path, copy_db_path)
#copy_db_path =  'structures/hexag_perovs_strained_copy.db'

files_folder = Path('NEB/Ba7Nb4MoO20_p1_e0/').resolve().as_posix()
name = 'Ba7Nb4MoO20_p1_neb'

with connect(old_db_path) as old_db:
    sys_row = old_db.get(name=name)
    sys_atoms = sys_row.toatoms()
    sys_kvps = sys_row.key_value_pairs
    with connect(new_db_path) as new_db:
        new_db.write(sys_atoms,sys_kvps)    



Then, modify the entry to match as best as possible with respect to the currrent scheme.

In [2]:

with connect(new_db_path) as db:
    sys_row = db.get(name=name)
    sys_id = sys_row.id
    sys_atoms = sys_row.toatoms()
    kvps = [kvp for kvp in sys_row]

    keys_values = {}
    keys_values['name'] = f"Ba7Nb4MoO20_p1_e0_neb"
    keys_values['dir'] = files_folder
    keys_values['dopant']=int(sys_row.name[-5])
    keys_values['in_plane'] = 1.00
    db.update(sys_id, **keys_values)



In [ ]:
from ase.db import connect
from pathlib import Path
import shutil

# Read and generate an atoms entry from the NEB run. Write/Update it to the DB.

new_db_path = 'structures/hexag_perovs_strained.db'
old_db_path = 'structures/hexag_perovs_wdiscards.db'
copy_db_path = 'structures/hexag_perovs_strained_copy.db'

db_path = shutil.copy(new_db_path, copy_db_path)



In [57]:
print(keys_values)

{'name': 'Ba7Nb4MoO20_p1_biaxial_e0_neb', 'dir': '/home/energy/armoma/phd/hex_perovs/strained/NEB/Ba7Nb4MoO20_p1_e0', 'forward_e': 1.077582239999856, 'delta_e': 0.5970954699998856, 'reverse_e': 0.4804867699999704, 'barrier': 1.077582239999856, 'neg_barrier': 'no', 'mask': 'biaxial', 'dopant': 1}


In [43]:
kvps = [kvp for kvp in sys_row]

In [44]:
sys_row.key_value_pairs

{'name': 'Ba7Nb4MoO20_p1_neb',
 'dir': '/home/scratch3/armoma/phd/hex_perovs/discarded/NEB/Ba7Nb4MoO20_p1',
 'forward_e': 1.077582239999856,
 'delta_e': 0.5970954699998856,
 'reverse_e': 0.4804867699999704,
 'barrier': 1.077582239999856,
 'neg_barrier': 'no'}

In [45]:
print(kvps)

['name', 'dir', 'forward_e', 'delta_e', 'reverse_e', 'barrier', 'neg_barrier', 'id', 'unique_id', 'ctime', 'mtime', 'user', 'numbers', 'positions', 'cell', 'pbc', 'initial_magmoms', 'calculator', 'calculator_parameters', 'energy', 'forces']


In [7]:
from ase.db import connect
from ase.db.row import atoms2dict
from ase.io import read
from pathlib import Path
import re
from ase.db.sqlite import SQLite3Database
from collections import Counter
from ase import data
from copy import copy
import shutil
import uuid  # For generating unique ids

# Read and generate an atoms entry from the NEB run. Write/Update it to the DB.

new_db_path = 'structures/hexag_perovs_strained.db'
copy_db_path =  'structures/hexag_perovs_strained_copy.db'
old_db_path = 'structures/hexag_perovs_wdiscards.db'

db_path = shutil.copy(new_db_path, copy_db_path)

files_folder = Path('NEB/Ba7Nb4MoO20_p1_e0/').resolve().as_posix()

with connect(old_db_path) as old_db:
    sys_row = old_db.get(name='Ba7Nb4MoO20_p1_neb')
    sys_atoms = sys_row.toatoms()
    sys_kvps = sys_row.key_value_pairs
    for mask in ['uniaxial', 'biaxial']:
        sys_kvps['name'] = f"Ba7Nb4MoO20_p1_{mask}_e0_neb"
        sys_kvps['dir'] = files_folder
        sys_kvps['mask'] = mask
        
        # Generate a new unique ID
        sys_kvps['unique_id'] = str(uuid.uuid4())  # Ensure the unique_id is different

        with connect(copy_db_path) as new_db:
            new_db.write(sys_atoms, key_value_pairs=sys_kvps)

ValueError: Bad key: unique_id

In [6]:
print(sys_kvps)

{'name': 'Ba7Nb4MoO20_p1_biaxial_e0_neb', 'dir': '/home/energy/armoma/phd/hex_perovs/strained/NEB/Ba7Nb4MoO20_p1_e0', 'forward_e': 1.077582239999856, 'delta_e': 0.5970954699998856, 'reverse_e': 0.4804867699999704, 'barrier': 1.077582239999856, 'neg_barrier': 'no', 'mask': 'biaxial'}


### Change the resources of the queued jobs to the epyc96 partition.

In [1]:
from perqueue.queue import PersistentQueue
from perqueue.selection import Selection
from ase.db import connect
from perqueue.task_classes.util_classes import Resources

# Get the entries from the active pq database and modify their resources
with PersistentQueue() as pq:
    entries = pq.get_entries()
    
s = Selection(states='q')
targets = s.filter(entries)

# Modify the resources of the entries
new_res = Resources.from_string("96:1:epyc96:50h")
with PersistentQueue() as pq:
    for en in targets:
        en._task.resources = new_res
        pq.save_resources(en.key, new_res, False)
        print(en)
    
        
    # pq_key = en.key
    # #print(en)
    # with PersistentQueue() as pq:
    #     if pq.get_code(pq_key).name != 'neb.py':
    #         print(f"New resources: {new_res1} for pq key: {pq_key}")
    #         pq.save_resources(pq_key, Resources(new_res1), all=False)
    #         print(en)
        #else:
        #    print(f"New resources: {new_res2} for pq key: {pq_key}")
        #    pq.save_resources(pq_key, Resources(new_res2), all=True)
        
        #pq.resubmit(en)
    
    #break

Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': -2.0}, resources=Resources(cores=96, processes=1, tmax=180000, nodename='epyc96')), entry_key=1, name='apply_strain.py', mq_id=7524397, state='q', err_txt=None, entry_group_key=1, pq_args=None, data=None)
Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': -1.0}, resources=Resources(cores=96, processes=1, tmax=180000, nodename='epyc96')), entry_key=4, name='apply_strain.py', mq_id=7524398, state='q', err_txt=None, entry_group_key=2, pq_args=None, data=None)
Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': 0.0}, resources=Resources(cores=96, processes=1, tmax=180000, nodename='epyc96')), entry_key=7, na

### Modify the state of an entry

In [1]:
from perqueue.queue import PersistentQueue
from perqueue.selection import Selection
from ase.db import connect
from perqueue.task_classes.util_classes import Resources
from perqueue.task_classes.util_classes import EntryState
import perqueue.coordinator as coord

# Get the entries from the active pq database and modify their resources
with PersistentQueue() as pq:
    entries = pq.get_entries()
    
s = Selection(ids=[12,32])
targets = s.filter(entries)

#new_state = EntryState('q')
with PersistentQueue() as pq:
    for en in targets:
        # Change the state of the entry
        coord.set_state(pq._backend, en.key, EntryState.succeeded, None)
        
        # Populate the data field in the entry
        #coord.set_data(pq._backend, en.key, {'db_id': 1})
        
        # Resubmit the entry
        #pq.resubmit(en)
        
        #break
        